# Model serving & deployment testing

## Abstract

This notebook serves the hypertuned model.

TODO: Update

The model to deploy is an XGBoost regressor with the following hyperparameters values:
- `max_depth` = 9
- `learning_rate` = 0.31619
- `n_estimators` = 767
- `subsample` = 1
- `gamma` = 0

With those hyperparameters values frozen, the model has already been trained on the full data available (_the original file `train.csv`_) during the [hypertuning phase](https://6fbe85b20ee1f013-dot-us-west1.notebooks.googleusercontent.com/lab/tree/Demo%20#2/5.%20Model%20hyper-parameter%20tuning%20&%20evaluation.ipynb).
<br>The model has been serialized in Cloud Storage in the __modelling_serializations__ folder.
<br>The model folder path is: `gs://szilard_aliz_sandbox/pierre_tasks/demo2/modelling_serializations_2/20200216_205755/to_deploy`.

Let's deploy this model and test it with an online prediction request feeding some sample input data taken from the original file `test.csv`.

## Serving the model

### Create endpoint

In [1]:
!gcloud ai endpoints create --region="europe-west3" --display-name="blackfriday"

Using endpoint [https://europe-west3-aiplatform.googleapis.com/]
Waiting for operation [3443004114147999744]...
.................................................................................................done.
Created Vertex AI endpoint: projects/53911330556/locations/europe-west3/endpoints/7113681902036320256.


In [2]:
!gcloud ai endpoints list --region="europe-west3" --filter=display_name="blackfriday"

ENDPOINT_ID          DISPLAY_NAME
7113681902036320256  blackfriday


Using endpoint [https://europe-west3-aiplatform.googleapis.com/]


In the following, we use the obtained `ENDPOINT_ID`.

### Create model

The model artifact's filename must exactly match `model.joblib`, if Joblib is used. There are other available formats, see documentation.

In [4]:
!gsutil cp "gs://aliz-ml-spec-2022/demo-2/modelling_serializations/20220528_184410/gamma=0.0__learning_rate=0.1__max_depth=3__n_estimators=100__n_jobs=-1__subsample=1.0__tree_method=hist.joblib" "gs://aliz-ml-spec-2022/demo-2/modelling_serializations/20220528_184410/model.joblib"

Copying gs://aliz-ml-spec-2022/demo-2/modelling_serializations/20220528_184410/gamma=0.0__learning_rate=0.1__max_depth=3__n_estimators=100__n_jobs=-1__subsample=1.0__tree_method=hist.joblib [Content-Type=application/octet-stream]...
/ [0 files][    0.0 B/214.9 KiB]                                                
/ [1 files][214.9 KiB/214.9 KiB]                                                

Operation completed over 1 objects/214.9 KiB.                                    


In [16]:
!gcloud ai models upload --region="europe-west3" --display-name="blackfriday" --container-image-uri="europe-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest" --artifact-uri="gs://aliz-ml-spec-2022/demo-2/modelling_serializations/20220528_184410/"

Using endpoint [https://europe-west3-aiplatform.googleapis.com/]
Waiting for operation [3021917548988858368]...
...................done.


In [17]:
!gcloud ai models list --region="europe-west3"

MODEL_ID             DISPLAY_NAME
3246180537659817984  blackfriday
6337901681849663488  blackfriday


Using endpoint [https://europe-west3-aiplatform.googleapis.com/]


### Deploy model

Deployment might take a while.

In [18]:
!gcloud ai endpoints deploy-model 7113681902036320256 --region=europe-west3 --model=3246180537659817984 --display-name="blackfriday" --machine-type=n1-standard-2 --min-replica-count=1 --max-replica-count=1 --traffic-split=0=100

Using endpoint [https://europe-west3-aiplatform.googleapis.com/]
Waiting for operation [8827057468669427712]...
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

#### Predict with Gcloud

In [ ]:
# TODO: genereate request.json

In [ ]:
# !gcloud ai endpoints predict 7113681902036320256 --region=europe-west3 --json-request=request.json

#### Predict with Python

In [ ]:
from google.cloud import aiplatform


def predict(
    project: str, location: str, instances: list, endpoint: str
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint(endpoint)

    prediction = endpoint.predict(instances=instances)
    print(prediction)
    return prediction

predict(
    project="aliz-ml-spec-2022-submission",
    location="europe-west3",
    instances=[
      [1000004, "P00128942", "M", "46-50", 7, "B", 2, 1, "1", "11.0", "nan"],
      [1000009, "P00113442", "M", "26-35", 17, "C", 0, 0, "3", "5.0", "nan"],
      [1000010, "P00288442", "F", "36-45", 1, "B", 4, 1, "5", "14.0", "nan"]
    ],
    endpoint="7113681902036320256"
)

#### Cleanup

In [14]:
!gcloud ai endpoints undeploy-model --region="europe-west3" --deployed-model-id=7113681902036320256

ERROR: (gcloud.ai.endpoints.undeploy-model) argument --region: ENDPOINT must be specified.
Usage: gcloud ai endpoints undeploy-model (ENDPOINT : --region=REGION) --deployed-model-id=DEPLOYED_MODEL_ID [optional flags]
  optional flags may be  --help | --region | --traffic-split

For detailed information on this command and its flags, run:
  gcloud ai endpoints undeploy-model --help


In [ ]:
!gcloud ai endpoints delete 7113681902036320256 --region="europe-west3"

In [15]:
!gcloud ai models delete 6337901681849663488 --region="europe-west3"

Using endpoint [https://europe-west3-aiplatform.googleapis.com/]
ERROR: (gcloud.ai.models.delete) FAILED_PRECONDITION: The Model "projects/53911330556/locations/europe-west3/models/6337901681849663488" is deployed or being deployed at the following Endpoint(s): projects/53911330556/locations/europe-west3/endpoints/7113681902036320256.
